# Intra-day Trading with Facebook Prophet

In [8]:
# Libraries
from eod import EodHistoricalData
import pandas as pd
from datetime import datetime, timedelta
import calendar
from tqdm import tqdm
import numpy as np
import random
import plotly.express as px
from prophet import Prophet

# Importing and assigning the api key
with open("../../eodHistoricalData-API.txt", "r") as f:
    api_key = f.read()
    
# EOD Historical Data client
client = EodHistoricalData(api_key)

# Getting Price Data

In [12]:
ago = datetime.utcnow() - timedelta(10)
timestamp_unix = str(calendar.timegm(ago.utctimetuple()))

In [111]:
resp = client.get_prices_intraday('BTC-USD.CC', interval='1h', from_=timestamp_unix)

In [116]:
def getIntradayPrices(crypto, n_days, training_amt, mov_avg):
    """
    Gets crypto prices from now to N days ago and training amount will be in addition 
    to the number of days to train. (According to the EOD documentation: The maximum 
    periods between ‘from’ and ‘to’ are 120 days for 1-minute interval, 600 days 
    for 5-minute interval and 7200 days for 1 hour interval.)
    """
    
    # Getting the date from N days ago
    ago = datetime.utcnow() - timedelta(n_days+training_amt)
    
    # Getting the unix timestamp format for the intraday data from the API
    timestamp_unix = str(calendar.timegm(ago.utctimetuple()))
    
    # Retrieving price data
    resp = client.get_prices_intraday(
        crypto+'-USD.CC', 
        interval="1h", 
        from_=timestamp_unix
    )
    
    # Removing the last time period for uniformity if necessary
    if resp[-1]['datetime'][-5:-3] != "00":
        resp = resp[:-1]
    
    # Price formatting for the dataframe
    prices = pd.DataFrame(resp)
        
    # Filling NaNs with the most recent values for any missing data
    prices = prices.fillna(method='ffill')
    
    # Getting the N Day Moving Average and rounding the values for some light data preprocessing
    prices['MA'] = prices[['open']].rolling(
        window=mov_avg
    ).mean().apply(lambda x: round(x, 6))

    # Resetting format for FBP
    prices = prices.rename(
        columns={"datetime": "ds", "MA": "y"}
    )
    
    return prices

In [117]:
df = getIntradayPrices(
    crypto="BTC", 
    n_days=10, 
    training_amt=2, 
    mov_avg=5
)

In [118]:
df.tail(10)

,timestamp,gmtoffset,ds,open,high,low,close,volume,y
272,1641492000,0,2022-01-06 18:00:00,43064.273437,43341.214843,43063.226562,43310.066406,160088064.0,43019.626562
273,1641495600,0,2022-01-06 19:00:00,43361.417968,43414.726562,43313.742187,43393.613281,160088064.0,43058.494531
274,1641499200,0,2022-01-06 20:00:00,43361.417968,43414.726562,43313.742187,43393.613281,160088064.0,43139.800000
275,1641502800,0,2022-01-06 21:00:00,43361.417968,43414.726562,43313.742187,43393.613281,160088064.0,43214.786718
276,1641506400,0,2022-01-06 22:00:00,43361.417968,43414.726562,43313.742187,43393.613281,160088064.0,43301.989062
277,1641510000,0,2022-01-06 23:00:00,43361.417968,43414.726562,43313.742187,43393.613281,160088064.0,43361.417968
278,1641513600,0,2022-01-07 00:00:00,43139.398437,43152.972656,42861.828125,42892.796875,57796608.0,43317.014062
279,1641517200,0,2022-01-07 01:00:00,42863.527343,43009.832031,42859.769531,42999.421875,214808576.0,43217.435937
280,1641520800,0,2022-01-07 02:00:00,42997.125000,42998.781250,42845.070312,42845.070312,62961664.0,43144.577343
281,1641524400,0,2022-01-07 03:00:00,42844.722656,42854.761718,42425.976562,42425.976562,297058304.0,43041.238281


# Time Series Analysis with Facebook Prophet

In [119]:
def fbpTrainPredict(df, forecast_period, freq):
    """
    Uses FB Prophet and fits to a appropriately formatted DF. Makes a prediction N days into 
    the future based on given forecast period. Returns predicted values as a DF.
    """
    # Setting up prophet
    m = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False
    )
    
    # Fitting to the prices
    m.fit(df[['ds', 'y']])
    
    # Future DF
    future = m.make_future_dataframe(
        periods=forecast_period,
        freq=freq,
        include_history=False
    )
        
    # Predicting values
    forecast = m.predict(future)

    # Returning a set of predicted values
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [120]:
fbpTrainPredict(df.tail(200), 5, 'H')

,ds,yhat,yhat_lower,yhat_upper
0,2022-01-07 04:00:00,42076.082845,41518.115683,42651.131848
1,2022-01-07 05:00:00,41970.341448,41395.091215,42568.723253
2,2022-01-07 06:00:00,41864.600052,41292.992691,42442.907005
3,2022-01-07 07:00:00,41758.858656,41201.650668,42362.965964
4,2022-01-07 08:00:00,41653.117259,41068.526982,42221.144928
